In [1]:
!git clone https://github.com/LizaDerb/probing_compositionality/

Cloning into 'probing_compositionality'...
remote: Enumerating objects: 248, done.
remote: Counting objects: 100% (23/23), done.
remote: Compressing objects: 100% (14/14), done.
remote: Total 248 (delta 13), reused 8 (delta 8), pack-reused 225
Receiving objects: 100% (248/248), 101.21 MiB | 13.30 MiB/s, done.
Resolving deltas: 100% (41/41), done.
Updating files: 100% (55/55), done.


# Russian AN и NN

## Датасет

In [2]:
import pandas as pd

In [37]:
corpus_AN = pd.read_csv('/content/probing_compositionality/data/rus/an_corpus.csv')
corpus_NN = pd.read_csv('/content/probing_compositionality/data/rus/nn_corpus.csv')

Лейблы:

*   Non-compositional (0)
*   Compositional (1)
*   Ambiguous (2)

Каждое выражение относится только к одной из групп.

In [38]:
corpus_AN.groupby('label')['idiom'].value_counts()

label  idiom                 
0      новый тип                 16
       налоговый кодекс          12
       уголовный кодекс          10
       огненная трасса           10
       абсолютное большинство    10
                                 ..
2      финансовое дело            3
       старая трасса              3
       столичный клуб             3
       существенное повышение     3
       следующая картина          3
Name: count, Length: 5591, dtype: int64

In [39]:
corpus_AN['label'].value_counts()

label
1    26548
0     1509
2     1309
Name: count, dtype: int64

In [40]:
corpus_NN.groupby('label')['idiom'].value_counts()

label  idiom            
0      точка зрения         220
       образ жизни           37
       рынок труда           24
       свобода слова         21
       душа населения        19
                           ... 
2      положение вещей        2
       эфир телеканалов       2
       эфир телекомпании      2
       народ моря             2
       движение тел           2
Name: count, Length: 7305, dtype: int64

In [41]:
corpus_NN['label'].value_counts()

label
1    22846
0      785
2      308
Name: count, dtype: int64

In [42]:
corpus = pd.concat([corpus_NN, corpus_AN])

Исключим ambiguous

In [43]:
corpus = corpus[corpus['label'] != 2]

In [44]:
corpus.info()

<class 'pandas.core.frame.DataFrame'>
Index: 51688 entries, 308 to 28056
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   idiom     51688 non-null  object
 1   label     51688 non-null  int64 
 2   sentence  51688 non-null  object
dtypes: int64(1), object(2)
memory usage: 1.6+ MB


Вручную разделим датасет на test и train, чтобы выражения в выборках не повторялись.

In [70]:
corpus[corpus['label'] == 0]['idiom'].value_counts()[:300].sum()

2130

In [71]:
non_compos = corpus[corpus['label'] == 0]['idiom'].value_counts().index[:300].tolist()
print(non_compos)

['точка зрения', 'образ жизни', 'рынок труда', 'свобода слова', 'душа населения', 'повестка дня', 'новый тип', 'банк крови', 'ремень безопасности', 'вертикаль власти', 'налоговый кодекс', 'культ личности', 'абсолютное большинство', 'огненная трасса', 'уголовный кодекс', 'лунная программа', 'свобода выбора', 'властелин колец', 'бассейн реки', 'конец света', 'большое значение', 'белый дом', 'боевые действия', 'волновой поток', 'временная администрация', 'второй секретарь', 'великая революция', 'великий пост', 'вечная мерзлота', 'властная вертикаль', 'бюджетный кодекс', 'большая деньги', 'крепостное право', 'красная армия', 'крайний случай', 'костный мозг', 'круглый стол', 'королевский институт', 'католическая церковь', 'широкая публика', 'квадратный метр', 'ключевой вопрос', 'комсомольский правда', 'большая пятерка', 'столовая ложка', 'холодная война', 'федеральный орган', 'учёная степень', 'узкий круг', 'военная операция', 'главный герой', 'главный инженер', 'главная роль', 'высокое кач

In [72]:
corpus[corpus['label'] == 1]['idiom'].value_counts()[:100].sum()

3470

In [73]:
compos = corpus[corpus['label'] == 1]['idiom'].value_counts().index[:100].tolist()
print(compos)

['заводская лаборатория', 'задняя поверхность', 'бывший министр', 'чемпионат мира', 'налоговый орган', 'вид спорта', 'российский медик', 'темп роста', 'тысяча рублей', 'рост цен', 'важный этап', 'крошечный станок', 'конституционное собрание', 'доктор наук', 'ультразвуковое колебание', 'городское поселение', 'тысяча человек', 'нештатная ситуация', 'решение проблем', 'миллион рублей', 'система образования', 'сахалинская область', 'незнакомый человек', 'глава государства', 'тысяча долларов', 'длинная история', 'круглосуточный режим', 'орган власти', 'миллион долларов', 'энергетический кризис', 'общее количество', 'большая потеря', 'миллиард рублей', 'миллиард долларов', 'нужный компонент', 'друг друга', 'медицинская организация', 'право человека', 'социалистический плюрализм', 'конкретная группа', 'единый учебник', 'основная группа', 'производительность труда', 'кожаный ботинок', 'длинная надпись', 'чемпион мира', 'принятие решений', 'современная бионика', 'решение задач', 'ленинская конц

In [74]:
phrase_train = non_compos[:150] + compos[:50]
phrase_test = non_compos[150:] + compos[50:]

In [75]:
corpus_train = corpus[corpus['idiom'].isin(phrase_train)]
corpus_test = corpus[corpus['idiom'].isin(phrase_test)]

In [76]:
print('train: ', len(corpus_train))
print('test: ', len(corpus_test))

train:  3791
test:  1809


In [77]:
stat_train = corpus_train['label'].value_counts().rename('num').to_frame()
stat_train['percentage'] = (stat_train.num * 100 / stat_train.num.sum()).round(1).astype(str) + '%'
stat_train

,num,percentage
label,,
1,2275,60.0%
0,1516,40.0%


In [78]:
stat_test = corpus_test['label'].value_counts().rename('num').to_frame()
stat_test['percentage'] = (stat_test.num * 100 / stat_test.num.sum()).round(1).astype(str) + '%'
stat_test

,num,percentage
label,,
1,1195,66.1%
0,614,33.9%


## RuBERT

In [ ]:
!pip install datasets

In [80]:
from datasets import Dataset

In [81]:
import seaborn as sns
from tqdm.notebook import tqdm
import numpy as np
import warnings
import random
import pickle

import torch
from torch.utils.data import DataLoader

from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, roc_auc_score, classification_report
from sklearn import preprocessing
from transformers import AutoTokenizer, AutoModel

In [82]:
train = Dataset.from_pandas(corpus_train)
test = Dataset.from_pandas(corpus_test)

In [83]:
train

Dataset({
    features: ['idiom', 'label', 'sentence', '__index_level_0__'],
    num_rows: 3791
})

In [84]:
test

Dataset({
    features: ['idiom', 'label', 'sentence', '__index_level_0__'],
    num_rows: 1809
})

In [85]:
rubert_tokenizer = AutoTokenizer.from_pretrained('DeepPavlov/rubert-base-cased')
rubert = AutoModel.from_pretrained('DeepPavlov/rubert-base-cased')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/642 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.65M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of the model checkpoint at DeepPavlov/rubert-base-cased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [86]:
torch.cuda.is_available()

True

In [87]:
device_cuda = torch.device("cuda")
device_cpu = torch.device("cpu")

In [88]:
def vectorize_dataset(data, tokenizer, model):
    model = model.to(device_cuda)
    res_cls = []
    res_mean = []
    for batch in tqdm(data):
        toks = tokenizer(batch['sentence'], padding='max_length', truncation=True, return_tensors='pt', max_length=300)
        with torch.no_grad():
            model_output = model(**{k: v.to(model.device) for k, v in toks.items()})
        res_cls.append(model_output.last_hidden_state[:,0,:]) #достаём вектора для CLS токенов
        res_mean.append(torch.mean(model_output.last_hidden_state, dim=1)) #среднее
    res_cls = torch.vstack(res_cls)
    res_mean = torch.vstack(res_mean)
    return res_cls.detach().cpu().numpy(), res_mean.detach().cpu().numpy()

In [89]:
dl_train = DataLoader(train, batch_size=256, shuffle=False,
                drop_last=False)
dl_test = DataLoader(test, batch_size=256, shuffle=False,
                drop_last=False)
vecs_train_cls, vecs_train_mean = vectorize_dataset(dl_train, rubert_tokenizer, rubert)
vecs_test_cls, vecs_test_mean = vectorize_dataset(dl_test, rubert_tokenizer, rubert)

  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/8 [00:00<?, ?it/s]

In [90]:
vectors = {'vecs_train_cls' : vecs_train_cls, 'vecs_train_mean' : vecs_train_mean,
           'vecs_test_cls' : vecs_test_cls, 'vecs_test_mean' : vecs_test_mean}

for name in vectors.keys():
    with open(f'{name}.pkl', 'wb') as file:
         pickle.dump(vectors[name], file)

In [91]:
vector_names = ['vecs_train_cls', 'vecs_train_mean', 'vecs_test_cls', 'vecs_test_mean']
vecs = []

for name in vector_names:
    with open(f'{name}.pkl', "rb") as fIn:
        vecs.append(pickle.load(fIn))

train_cls = vecs[0]
train_mean = vecs[1]
test_cls = vecs[2]
test_mean = vecs[3]

In [92]:
train_cls.shape

(3791, 768)

In [93]:
test_mean.shape

(1809, 768)

In [94]:
train_labels = train['label']
test_labels = test['label']

In [95]:
def calculate_metrics(true_labels, predictions):
    matrix = confusion_matrix(true_labels, predictions)
    acc = accuracy_score(true_labels, predictions)
    prec = precision_score(true_labels, predictions)
    rec = recall_score(true_labels, predictions)
    f1 = f1_score(true_labels, predictions)
    roc_auc = roc_auc_score(true_labels, predictions)
    return matrix, acc, prec, rec, f1, roc_auc

In [96]:
def baseline_rand_vec(vecs):
    size = vecs.shape[1]
    num_vecs = vecs.shape[0]
    rand_vecs = []
    for _ in range(num_vecs):
        rand_vecs.append(np.random.uniform(-5.0826163,1.5603778,size))
    return rand_vecs

In [97]:
def baseline_rand_pred(vecs, labels):
    num_vecs = vecs.shape[0]
    rand_vecs = []
    rand_pred = np.array(range(num_vecs), dtype=int)
    for i in range(0, num_vecs):
        rand_pred[i] = random.choice(labels)
    return rand_pred

In [98]:
def rand_pred_func(vecs, labels, true_labels, n=50):
    results_rand_pred = []
    for _ in range(n):
        pred_labels = baseline_rand_pred(vecs, labels)
        results_rand_pred.append(calculate_metrics(true_labels, pred_labels))
    res_rand_pred = pd.DataFrame(results_rand_pred, columns=['matrix', 'accuracy', 'precision', 'recall', 'f1', 'roc_auc'])
    res_rand_pred.to_csv('res_rand_pred.csv')

In [99]:
def get_results(train_data, train_labels, test_data, test_labels):
    results_train = []
    results_test = []
    for n in tqdm(range(50)):
        clf = MLPClassifier(random_state=n, max_iter=300, verbose=1)
        clf.fit(train_data, train_labels)
        train_pred = clf.predict(train_data)
        test_pred = clf.predict(test_data)
        results_train.append(calculate_metrics(train_labels, train_pred))
        results_test.append(calculate_metrics(test_labels, test_pred))
    return results_train, results_test, test_pred

In [100]:
labels = [0, 1]

In [101]:
rand_pred_func(test_cls, labels, test_labels)

In [ ]:
rand_vecs = baseline_rand_vec(train_cls)
results_train_rand, results_test_rand, test_pred_rand = get_results(rand_vecs, train_labels, test_cls, test_labels)
res_test_rand = pd.DataFrame(results_test_rand, columns=['matrix', 'accuracy', 'precision', 'recall', 'f1', 'roc_auc'])
res_train_rand = pd.DataFrame(results_train_rand, columns=['matrix', 'accuracy', 'precision', 'recall', 'f1', 'roc_auc'])
res_test_rand.to_csv('res_test_rand_vec.csv')
res_train_rand.to_csv('res_train_rand_vec.csv')

In [ ]:
results_train_mean, results_test_mean, test_pred_mean = get_results(train_mean, train_labels, test_mean, test_labels)
res_test_mean = pd.DataFrame(results_test_mean, columns=['matrix', 'accuracy', 'precision', 'recall', 'f1', 'roc_auc'])
res_train_mean = pd.DataFrame(results_train_mean, columns=['matrix', 'accuracy', 'precision', 'recall', 'f1', 'roc_auc'])
res_test_mean.to_csv('res_test_mean.csv')
res_train_mean.to_csv('res_train_mean.csv')

In [107]:
print(classification_report(test_labels, test_pred_mean))

              precision    recall  f1-score   support

           0       0.43      0.68      0.53       614
           1       0.76      0.54      0.63      1195

    accuracy                           0.59      1809
   macro avg       0.60      0.61      0.58      1809
weighted avg       0.65      0.59      0.60      1809



In [ ]:
results_train_cls, results_test_cls, test_pred_cls = get_results(train_cls, train_labels, test_cls, test_labels)
res_test_cls = pd.DataFrame(results_test_cls, columns=['matrix', 'accuracy', 'precision', 'recall', 'f1', 'roc_auc'])
res_train_cls = pd.DataFrame(results_train_cls, columns=['matrix', 'accuracy', 'precision', 'recall', 'f1', 'roc_auc'])
res_test_cls.to_csv('res_test_cls.csv')
res_train_cls.to_csv('res_train_cls.csv')

In [108]:
print(classification_report(test_labels, test_pred_cls))

              precision    recall  f1-score   support

           0       0.41      0.55      0.47       614
           1       0.72      0.60      0.65      1195

    accuracy                           0.58      1809
   macro avg       0.57      0.57      0.56      1809
weighted avg       0.62      0.58      0.59      1809



In [110]:
!zip -r /content/file.zip /content/data

updating: content/data/ (stored 0%)
updating: content/data/vecs_train_mean.pkl (deflated 9%)
updating: content/data/vecs_train_cls.pkl (deflated 8%)
updating: content/data/res_test_rand_vec.csv (deflated 69%)
updating: content/data/res_train_mean.csv (deflated 66%)
updating: content/data/vecs_test_mean.pkl (deflated 8%)
updating: content/data/res_test_mean.csv (deflated 57%)
updating: content/data/res_rand_pred.csv (deflated 60%)
updating: content/data/res_train_rand_vec.csv (deflated 92%)
updating: content/data/vecs_test_cls.pkl (deflated 7%)
  adding: content/data/res_train_cls.csv (deflated 64%)
  adding: content/data/res_test_cls.csv (deflated 57%)
